### Кошман Дмитрий
## Теория к задаче Переезд

### Алгоритм решения

Основная идея состоит в том, чтобы бинарным поиском пройти по возможным минимальным количествам слитков у самого загруженного человека, и на каждый вопрос вида "Можно ли так распределить слитки, чтобы у всех было не больше n слитков?" искать ответ, находя максимальный поток.

1. Бинарный поиск

Берем за нижнюю границу `low`:=0, за верхнюю `high` - максимальное количество слитков среди компании, и задаем вопрос, можно ли так распределить слитки, чтобы у всех было не больше, чем `mid` =(`low` + `high`) / 2 слитков? Если да, `high` = `mid`, иначе `low` = `mid` + 1. Продолжаем, пока `low` < `high`, после чего возвращаем в качестве ответа `low`.

2. Поток

Нам нужно ответить на вопрос "Можно ли так распределить слитки, чтобы у всех было не больше `target` слитков?". Сведем задачу к потоку. Пусть $s$ - источник, $t$ - сток, $v_i$ - вершина, отвечающая человеку из компании, $g_i$ - начальное количество слитков у каждого человека. Рассмотрим ориентированный полносвязный граф на вершинах $V = \{s, t\} \cup \{v_i\}$. Определим ограничения $c : V\times V \rightarrow \mathbb{Z}_+$ как:

$$
c(u,v) = \begin{cases}
	\max(0, g_i - \text{target}), & u=s, v=v_i \\
    \max(0, \text{target} - g_i), & u=v_i, v=t \\
    \infty, & u=v_i, v=v_j, v_i \text{ доверяет } v_j \\
	0, & \text{иначе} \\
\end{cases}
$$

То есть поток на ребре между $v_i, v_j$ можно интерпретировать как то, сколько слитков один человек передает другому, причем благодаря ограничениям на начальных и конечных ребрах, каждый может отдать не больше, чем у него в излишке по отношению к `target`, а получить не больше, чем ему не хватает до `target`.

С помощью алгоритма Форда-Фалкерсона находим максимальный поток $f : V\times V \rightarrow \mathbb{Z}$.

Проходим по вершинам $v_i$, и находим максимум итогового количества слитков у каждого: $g_i -f(s, v_i) + f(v_i, t)$. Если максимум не больше `target`, отвечаем на вопрос да, иначе нет.

### Доказательство правильности алгоритма

Почему, если алгоритм дал ответ `answer`, то существует такое распределение слитков, что у каждого будет не больше `answer` слитков, и меньшего распределения не существует?

Предположим, что этап алгоритма с потоком дает верный ответ. Тогда шаг бинарного поиска сохраняет инвариант, что существует распределение, что у всех будет не меньше `low` и не больше `high` слитков, причем меньше `low` распределения не может быть. Действительно, это верно при инициализации - в качестве итогового распределения слитков можно просто взять начальное. Пусть это было верно до шага, и поток говорит, что существует распределение, отвечающее `mid` или меньше, то это будет верно и после шага, если `mid` присвоить верхней границе. А если поток говорит, что не существует распределения, меньшего или равного `mid`, причем мы знаем, что существует распределение, отвечающее какому-то значению на интервале, то значит нужно искать его среди тех, которые больше `mid`, то есть как минимум `mid` + 1, и опять инвариант сохранится. Когда `low` станет равен `high`, то `low` благодаря инварианту будет искомым ответом.

Почему этап алгоритма с потоком дает верный ответ? Предположим обратное: пусть существует распределение, меньшее или равное `target`, а алгоритм говорит, что нет. Но это противоречит максимальности потока, который возвращает алгоритм Форда-Фалкерсона. Действительно, объем потока равен сумме потоков на ребрах, исходящих из источника. И распределение слитков, меньшее или равное `target`, и только такое распределение, насыщает все ребра из источника, и при этом удовлетворяет ограничениям на остальных ребрах, значит оно соответствует максимальному потоку, и алгоритм его найдет. И наоборот, если этап алгоритма с потоком говорит, что существует распределение, меньшее или равное `target`, то его можно предъявить: следуя положительным потокам на ребрах, люди могут передавать слитки, причем только тем, кому доверяют, при этом игнорируя ребра из источника и в сток. Благодаря свойствам потока, $\forall v_i:  \sum_{v} f(v_i, v) = 0 = \sum_{j} f(v_i, v_j) + f(v_i, s) + f(v_i, t)= \sum_{j} f(v_i, v_j) - f(s, v_i) + f(v_i, t)$. Заметим, что $\sum_{j} f(v_i, v_j)$ - это то, сколько $v_i$ в сумме передал, значит в итоге у него осталось $g_i - \sum_{j} f(v_i, v_j) = g_i - f(s, v_i) + f(v_i, t)$, а поскольку на последнем шаге алгоритма мы сравниваем именно это значение, то оно меньше или равно `target`.

### Временная сложность

Пусть людей всего $N$, доверительных пар $K$, а изначально слитков у каждого не больше $G$.
Сначала строим граф за $O(N + K)$, затем запускаем бинарный поиск. Шагов бинарного поиска будет порядка $\log G$, в каждом шаге запускаем алгоритм Форда-Фалкерсона, который работает за $O(GNK)$, и в конце проходим по всем вершинам за $O(N)$, чтобы найти максимальное количество слитков.

В итоге получаем временную сложность $O(N + K) + O((GNK + N)\log G)= O(GNK\log G)$

### Затраты памяти

Для графа нужно $O(N + K)$ памяти, и если хранить в памяти только потоки, соответствующие ребрам с ненулевыми ограничениями, и запускать алгоритм Форда-Фалкерсона на одном и том же графе, на ходу вычисляя ограничения, исходя из текущего `target`, то дополнительная память понадобится только для хранения потоков на ребрах, которых порядка $O(N + K)$, и для текущего ненулевого пути, который не больше $N$.

В итоге получаем затраты памяти $O(N + K)$.